## 10-1 convolution

In [7]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
# for reproducibility
#torch.manual_seed(777)
#if device == 'cuda':
    #torch.cuda.manual_seed_all(777)

In [10]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [11]:
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [12]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

### Conv2d(input_channel, output_channel, kernel_size)

### MaxPool2d(kernel_size)

#### - input(batch_size, channel, H, W)
#### - output(batch_size, channel, H, W)

![title](conv.png)

![title](cnn.png)

In [13]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 input = (N, 1, 28, 28)
        # Conv2d() -> (N, 32, 28, 28)
        # MaxPool2d -> (N, 32, 14, 14)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(N, 32,14, 14)
        # Conv2d() -> (N, 64, 14, 14)
        # MaxPool2d -> (N, 64, 7, 7)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  
        out = self.fc(out)
        return out

In [14]:
model = CNN().to(device)

In [15]:
criterion = torch.nn.CrossEntropyLoss().to(device)   
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>3}]\tcost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:   1]	cost = 0.211775705
[Epoch:   2]	cost = 0.0586491376
[Epoch:   3]	cost = 0.0433287881
[Epoch:   4]	cost = 0.0353688821
[Epoch:   5]	cost = 0.0285785887
[Epoch:   6]	cost = 0.0241733603
[Epoch:   7]	cost = 0.0193855707
[Epoch:   8]	cost = 0.0161962155
[Epoch:   9]	cost = 0.0138026522
[Epoch:  10]	cost = 0.0115011614
[Epoch:  11]	cost = 0.00975401141
[Epoch:  12]	cost = 0.0079246005
[Epoch:  13]	cost = 0.00786993559
[Epoch:  14]	cost = 0.00580927217
[Epoch:  15]	cost = 0.00570907816
Learning Finished!


In [17]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\Moon\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:60: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\Moon\Anaconda3\lib\site-packages\torchvision\datasets\mnist.py:50: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9868999719619751
